# Script para testear los modelos generados y visualizar el resultado en imágenes



In [ ]:
# comprobar que estamos utilizando GPU
!nvidia-smi
# !nvcc -V
# !nvidia-smi -q

# Instalar librerías no preinstaladas en Colab

In [1]:
!pip install imutils --quiet

# Conexión a drive

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
path_drive = '/content/gdrive/My Drive/Focusondriving'

Mounted at /content/gdrive


In [3]:
import os
from imutils import paths
import cv2
import numpy as np
import time
import matplotlib.pyplot as plt
import pandas as pd
import heapq
from statistics import mean


from keras.models import Model
from keras.models import load_model 
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing import image
from keras.optimizers import Adam
from keras.layers import Dense, Dropout, Flatten

# Función para crear la estructura del modelo para luego poder cargar los pesos de los modelos generados

In [4]:
def create_model(input_shape, n_classes, optimizer='rmsprop', fine_tune=0, n_model=1):
    """
    Compiles a model integrated with VGG16 pretrained layers
    
    input_shape: tuple - the shape of input images (width, height, channels)
    n_classes: int - number of classes for the output layer
    optimizer: string - instantiated optimizer to use for training. Defaults to 'RMSProp'
    fine_tune: int - The number of pre-trained layers to unfreeze.
                If set to 0, all pretrained layers will freeze during training
    """
    
    # Pretrained convolutional layers are loaded using the Imagenet weights.
    # Include_top is set to False, in order to exclude the model's fully-connected layers.
    if n_model == 4:
        conv_base = VGG19(include_top=False,
                     weights='imagenet', 
                     input_shape=input_shape)
    else:
        conv_base = VGG16(include_top=False,
                         weights='imagenet', 
                         input_shape=input_shape)
    
    # Defines how many layers to freeze during training.
    # Layers in the convolutional base are switched from trainable to non-trainable
    # depending on the size of the fine-tuning parameter.
    if fine_tune > 0:
        for layer in conv_base.layers[:-fine_tune]:
            layer.trainable = False
    else:
        for layer in conv_base.layers:
            layer.trainable = False

    # Create a new 'top' of the model (i.e. fully-connected layers).
    # This is 'bootstrapping' a new top_model onto the pretrained layers.
    top_model = conv_base.output
    top_model = Flatten(name="flatten")(top_model)
    if n_model == 1 or n_model == 2:
      top_model = Dense(4096, activation='relu')(top_model)
      top_model = Dense(1072, activation='relu')(top_model)
    
    if n_model == 3 or n_model == 4:
      top_model = Dense(4096, activation='relu')(top_model)
      top_model = Dense(1024, activation='relu')(top_model)
      top_model = Dense(256, activation='relu')(top_model)
      top_model = Dense(64, activation='relu')(top_model)
    
    if n_model == 5:
      top_model = Dense(4096, activation='relu')(top_model)
      top_model = Dense(2048, activation='relu')(top_model)
      top_model = Dense(1024, activation='relu')(top_model)
      top_model = Dense(512, activation='relu')(top_model)
      top_model = Dense(256, activation='relu')(top_model)
      top_model = Dense(128, activation='relu')(top_model)
      top_model = Dense(64, activation='relu')(top_model)
      top_model = Dense(32, activation='relu')(top_model)


    top_model = Dropout(0.2)(top_model)
    output_layer = Dense(n_classes, activation='softmax')(top_model)
    
    # Group the convolutional base and new fully-connected layers into a Model object.
    model = Model(inputs=conv_base.input, outputs=output_layer)

    # Compiles the model for training.
    model.compile(optimizer=optimizer, 
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

# Función que accede a la carpeta test y testea el modelo foto a foto

In [5]:
font = cv2.FONT_HERSHEY_SIMPLEX
fontScale = 1
fontColor = (255, 0, 0)
lineType = 2

In [62]:
def test_folder(test_data, model_name, model):
    directories = sorted(os.listdir(test_data))
    df = pd.DataFrame()
    for directory in directories:
        if '_' in directory:
            continue
        _dir = f'{test_data}/{directory}'
        len_files = len(os.listdir(_dir))
        category = _dir.split('/')[-1]
        test_name = _dir.split('/')[-2]
        print(f'{test_name} category {category} {len_files} files')
        img_list = paths.list_images(_dir)
        for i, img_path in enumerate(img_list):
            print(f"{directory} {img_path}")
            if i > 2:
              break
            x = preprocess_img(img_path)
            preds = model.predict(x)
            preds = list(preds[0])

            list_desc_order = heapq.nlargest(2, range(len(preds)), key=preds.__getitem__)

            result1 = f'c{list_desc_order[0]}'
            result2 = '-'
            result2_ = 0
            if preds[list_desc_order[1]] > 0.3:
                result2 = f'c{list_desc_order[1]}'
                result2_  = round(preds[list_desc_order[1]], 2)
            txt = f"category {directory} result 1 {result1} {round(preds[list_desc_order[0]],2)} | result2 {result2} {result2_}"
            txt = f"categoria {directory}"
            print(txt)
            score = round(preds[list_desc_order[0]], 2)*100
            score = int(score)
            txt2 = f"resultado {result1} probabilidad {score}%"
            img = cv2.imread(img_path)
            
            img = cv2.rectangle(img, pt1=(0, 380), pt2=(550, 480), color=(255, 255,255), thickness=-1)
            img = cv2.putText(img, txt, (20, 410), font, fontScale, (0, 0, 0), lineType)
            img = cv2.putText(img, txt2, (20, 450), font, fontScale, (255, 0, 0), lineType)
            img_name = img_path.split('/')[-1]
            img_name = f'{path_drive}/aws/img_results/cat_{directory}_res_{result1}_{img_name}'
            print(img_name)
            cv2.imwrite(img_name, img)
            correct2 = 0
            correct = 1 if result1 == directory or result2 == directory else 0
            if directory == 'c9':
                correct2 = 1 if result1 == 'c9' or result1 == 'c0' else 0
            error = 1 if correct == 0 else 0

            df = df.append(pd.DataFrame({'path': [img_path],
                                         'name': [class_dict[directory]],
                                         'category': [directory],
                                         'result1': [result1],
                                         'result1%': [preds[list_desc_order[0]]],
                                         'result2': [result2],
                                         'result2%': [result2_],
                                         'correct': [correct],                                    
                                         'error': [error],
                                         'correct2': [correct2],}), ignore_index=True)

            plt.imshow(img)
            plt.show()

    return resume(df, test_name, model_name)

In [63]:
def preprocess_img(img_path):
  img = image.load_img(img_path, target_size=(224, 224))
  x = image.img_to_array(img) 
  x = np.expand_dims(x, axis=0)
  x = preprocess_input(x)
  return x

# Testeo de modelos con diferentes conjuntos de test

In [65]:
input_shape = (224, 224, 3)
n_classes=10
optim_2 = Adam(learning_rate=0.0001)

test_model = {
    #'model1': f'{path_drive}/aws/model/Definitivo/model1.hdf5',
    #'model2': f'{path_drive}/aws/model/Definitivo/model2.hdf5',
    'model3': f'{path_drive}/aws/model/Definitivo/model3.hdf5',
    #'model4': f'{path_drive}/aws/model/Definitivo/model4.hdf5',
    #'model5': f'{path_drive}/aws/model/Definitivo/model5.hdf5'
    }
test_folders = [f'{path_drive}/aws/test',  f'{path_drive}/aws/test_henry_imanol']
test_folders = [f'{path_drive}/aws/test']


In [66]:
class_dict = {
    'c0': 'hands on the wheel',
    'c1': 'mobile in right hand',
    'c2': 'talking on the phone with right hand',
    'c3': "mobile in left hand",
    'c4': 'talking on the phone with left hand',
    'c5': 'touching at the dash',
    'c6': 'drinking',
    'c7': 'reaching behind',
    'c8': 'touching the head',
    'c9': 'looking to the side'
}

In [67]:
for i, test_data in enumerate(test_folders):
  print(f'{1} test folder {test_data}')
  for key, path_model in test_model.items():
      print(f"test model: {path_model.split('/')[-1]}")
      model = create_model(input_shape, n_classes, optim_2, fine_tune=2, modelo=3)
      model.load_weights(path_model)
      df_resume_test = test_folder(test_data, model_name=f'model{key}', model=model)
      print(df_resume_test)

1 test folder /content/gdrive/My Drive/Focusondriving/aws/test
test model: model3.hdf5
test category c0 114 files
c0 /content/gdrive/My Drive/Focusondriving/aws/test/c0/img_100313.jpg
categoria c0
/content/gdrive/My Drive/Focusondriving/aws/img_results/cat_c0_res_c1_img_100313.jpg
c0 /content/gdrive/My Drive/Focusondriving/aws/test/c0/img_100310.jpg
categoria c0
/content/gdrive/My Drive/Focusondriving/aws/img_results/cat_c0_res_c0_img_100310.jpg
c0 /content/gdrive/My Drive/Focusondriving/aws/test/c0/img_100315.jpg
categoria c0
/content/gdrive/My Drive/Focusondriving/aws/img_results/cat_c0_res_c0_img_100315.jpg
c0 /content/gdrive/My Drive/Focusondriving/aws/test/c0/img_100839.jpg
test category c1 104 files
c1 /content/gdrive/My Drive/Focusondriving/aws/test/c1/img_100296.jpg
categoria c1
/content/gdrive/My Drive/Focusondriving/aws/img_results/cat_c1_res_c1_img_100296.jpg
c1 /content/gdrive/My Drive/Focusondriving/aws/test/c1/img_100284.jpg
categoria c1
/content/gdrive/My Drive/Focusondr

# Crear un gif con los resultados y guardar en drive

In [70]:
import imageio
from imutils import paths
path_imgs = f"{path_drive}/aws/img_results"
path_save_gif = f"{path_drive}/aws/result.gif"
images = [imageio.imread(filename) for filename in paths.list_images(path_imgs)]
kargs = {'duration': 0.75}
imageio.mimsave(path_save_gif, images, **kargs)
print('gif created')

gif created
